# ✏️ Basic Setting

## 🔎 Importing Libraries

In [ ]:
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer, label_binarize
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from lightgbm import LGBMClassifier
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
from sklearn.metrics import confusion_matrix, accuracy_score
import matplotlib.pyplot as plt
from bayes_opt import BayesianOptimization
from sklearn.utils.class_weight import compute_class_weight
import warnings
import datetime

In [ ]:
'''%cd /content/
#lgbm with gpu on colab
!git clone https://github.com/Microsoft/LightGBM
%cd /content/LightGBM
!mkdir build
!cmake -DUSE_GPU=1
!make -j$(nproc)
!sudo apt-get -y install python-pip
!sudo -H pip install setuptools pandas numpy scipy scikit-learn -U
%cd /content/LightGBM/python-package
!sudo python setup.py install'''

'%cd /content/\n#lgbm with gpu on colab\n!git clone https://github.com/Microsoft/LightGBM\n%cd /content/LightGBM\n!mkdir build\n!cmake -DUSE_GPU=1\n!make -j$(nproc)\n!sudo apt-get -y install python-pip\n!sudo -H pip install setuptools pandas numpy scipy scikit-learn -U\n%cd /content/LightGBM/python-package\n!sudo python setup.py install'

In [ ]:
'''!pip install bayesian-optimization'''

  Created wheel for bayesian-optimization: filename=bayesian_optimization-1.2.0-cp36-none-any.whl size=11685 sha256=feebf53c5e87f200968d50262ef01b3e632ee5029a1a0f11f4cfbff5188741b4
  Stored in directory: /root/.cache/pip/wheels/5a/56/ae/e0e3c1fc1954dc3ec712e2df547235ed072b448094d8f94aec
Successfully built bayesian-optimization


# ✏️ Hyperparameter tuning (Bayesian Optimization)

## 🔎 Samsung Card version

In [ ]:
def LGB_opt(max_depth,
           gamma,
           min_child_weight,
           max_delta_step,
           subsample,
           colsample_bytree,
           learning_rate,
         ):

    global AUCbest
    global ITERbest

    folds = 10
    cv_score = 0

    lgb = LGBMClassifier(class_weight=None,
                        colsample_bytree = max(min(colsample_bytree, 1), 0), 
                        learning_rate = learning_rate,
                        #gamma = gamma, 
                        max_depth = int(max_depth), 
                        min_child_weight = min_child_weight, 
                        subsample = max(min(subsample, 1), 0),
                        num_leaves = min(2**int(max_depth),131072),
                        n_estimators = 20000, 
                        n_jobs=-1, 
                        device='gpu',
                        random_state=42)
                        
    lgb.fit(X_train, y_train,
            verbose=False,  
            early_stopping_rounds = 100,
            eval_set=[(X_test, y_test)],
            eval_metric=custom_eval)

    val_score = lgb.evals_result_['valid_0']['roc_auc'][-1]
    print(' Stopped after %d iterations with val-auc = %f val-gini = %f' % ( len(lgb.evals_result_['valid_0']['roc_auc']), val_score, (val_score*2-1)) )
    if ( val_score > AUCbest ):
        AUCbest = val_score
        ITERbest = len(lgb.evals_result_['valid_0']['roc_auc'])

    return (val_score*2) - 1

In [ ]:
def custom_eval(y_true, y_pred):
  preds = y_pred.reshape(11, -1).T
  lb = LabelBinarizer()
  lb.fit(y_true)
  label = lb.transform(y_true)
  return 'roc_auc' , roc_auc_score(label, preds), True

In [ ]:
LGB_BO = BayesianOptimization(LGB_opt, {
                                    'max_depth': (2, 20),
                                    'gamma': (0.001, 10.0),
                                    'min_child_weight': (0, 20),
                                    'max_delta_step': (0, 10),
                                    'subsample': (0.4, 1.0),
                                    'colsample_bytree' :(0.4, 1.0),
                                    'learning_rate' : (0.01, 0.1)
                                    })

In [ ]:
%cd /content/drive/My Drive/samsung_card/preprocess  

path = '../data/'
test_file = 'df_merged.csv'
df = pd.read_csv(path+test_file, index_col='cst_id_di')

/content/drive/My Drive/samsung_card/preprocess


In [ ]:
X = df.iloc[:, 1:].values; y = df.iloc[:, 0].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=1234)

In [ ]:
AUCbest = -1.
ITERbest = 0

with warnings.catch_warnings():
    warnings.filterwarnings('ignore')
    LGB_BO.maximize(init_points=2, n_iter=30, acq='ei', xi=0.0)

|   iter    |  target   | colsam... |   gamma   | learni... | max_de... | max_depth | min_ch... | subsample |
-------------------------------------------------------------------------------------------------------------
 Stopped after 166 iterations with val-auc = 0.883284 val-gini = 0.766569
|  1        |  0.7666   |  0.7363   |  1.363    |  0.07834  |  4.224    |  10.48    |  3.574    |  0.6068   |
 Stopped after 133 iterations with val-auc = 0.876236 val-gini = 0.752472
|  2        |  0.7525   |  0.4192   |  6.497    |  0.08306  |  3.924    |  14.8     |  1.103    |  0.637    |
 Stopped after 1475 iterations with val-auc = 0.887828 val-gini = 0.775655
|  3        |  0.7757   |  0.4664   |  2.01     |  0.01934  |  9.227    |  3.089    |  1.592    |  0.9715   |
 Stopped after 238 iterations with val-auc = 0.887142 val-gini = 0.774285
|  4        |  0.7743   |  0.6266   |  2.738    |  0.07024  |  7.605    |  10.07    |  14.75    |  0.8221   |
 Stopped after 2830 iterations with val-auc

## 🔎 Titanic Version

In [ ]:
titanic = pd.read_csv("titanic.csv")
titanic['Sex'] = titanic['Sex'].map({"male":0,'female':1})
titanic = titanic[['Survived', 'Pclass','Sex', 'Age', 'SibSp', 'Parch']]
titanic = titanic.fillna(round(titanic['Age'].mean()))
titanic['Age'] = titanic['Age'].astype(int)

In [ ]:
X = titanic.iloc[:, 1:]; y = titanic.iloc[:, 0]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
def LGB_opt(max_depth,
           gamma,
           min_child_weight,
           max_delta_step,
           subsample,
           colsample_bytree,
           learning_rate,
         ):

    global AUCbest
    global ITERbest

    folds = 10
    cv_score = 0

    lgb = LGBMClassifier(class_weight=None,
                        colsample_bytree = max(min(colsample_bytree, 1), 0), 
                        learning_rate = learning_rate,
                        #gamma = gamma, 
                        max_depth = int(max_depth), 
                        min_child_weight = min_child_weight, 
                        subsample = max(min(subsample, 1), 0),
                        num_leaves = min(2**int(max_depth),131072),
                        n_estimators = 20000, 
                        n_jobs=-1, 
#                         device='gpu',
                        random_state=42)
                        
    lgb.fit(X_train, y_train,
            verbose=False,  
            early_stopping_rounds = 100,
            eval_set=[(X_test, y_test)],
            eval_metric=['auc'])

    val_score = lgb.evals_result_['valid_0']['auc'][-1]
    print(' Stopped after %d iterations with val-auc = %f val-gini = %f' % ( len(lgb.evals_result_['valid_0']['auc']), val_score, (val_score*2-1)) )
    if ( val_score > AUCbest ):
        AUCbest = val_score
        ITERbest = len(lgb.evals_result_['valid_0']['auc'])

    return (val_score*2) - 1

In [ ]:
LGB_BO = BayesianOptimization(LGB_opt, {
                                    'max_depth': (2, 20),
                                    'gamma': (0.001, 10.0),
                                    'min_child_weight': (0, 20),
                                    'max_delta_step': (0, 10),
                                    'subsample': (0.4, 1.0),
                                    'colsample_bytree' :(0.4, 1.0),
                                    'learning_rate' : (0.01, 0.1)
                                    })

In [ ]:
AUCbest = -1.
ITERbest = 0

with warnings.catch_warnings():
    warnings.filterwarnings('ignore')
    LGB_BO.maximize(init_points=2, n_iter=30, acq='ei', xi=0.0)

|   iter    |  target   | colsam... |   gamma   | learni... | max_de... | max_depth | min_ch... | subsample |
-------------------------------------------------------------------------------------------------------------
 Stopped after 140 iterations with val-auc = 0.873810 val-gini = 0.747619
|  1        |  0.7476   |  0.9522   |  5.931    |  0.08519  |  1.429    |  9.256    |  5.171    |  0.848    |
 Stopped after 210 iterations with val-auc = 0.880695 val-gini = 0.761390
|  2        |  0.7614   |  0.9933   |  9.006    |  0.06377  |  2.197    |  2.862    |  8.896    |  0.7947   |
 Stopped after 159 iterations with val-auc = 0.866795 val-gini = 0.733591
|  3        |  0.7336   |  0.4893   |  8.692    |  0.09656  |  2.31     |  2.747    |  8.883    |  0.7896   |
 Stopped after 106 iterations with val-auc = 0.847555 val-gini = 0.695109
|  4        |  0.6951   |  0.7726   |  6.578    |  0.02408  |  6.739    |  5.268    |  18.02    |  0.5937   |
 Stopped after 154 iterations with val-auc =

In [ ]:
print(LGB_BO.max)

{'target': 0.781981981981982, 'params': {'colsample_bytree': 0.8161923815519119, 'gamma': 2.100636409041735, 'learning_rate': 0.02254700212192725, 'max_delta_step': 8.474820710132052, 'max_depth': 10.19077333497031, 'min_child_weight': 7.728551995575659, 'subsample': 0.4674282842643168}}
